# Tests with Numba

In [6]:
from numba import jit
import numpy as np
import time


def H(arr):
    x = np.roll(arr,1,axis=0)
    y = np.roll(arr,1,axis=1)
    return -np.sum(arr*(x+y))

@jit(nopython=True)
def H_nonp(arr):
    s=0
    w=len(arr)
    h=len(arr[0])
    for x in range(w):
        for y in range(h):
            s-=arr[x][y]*arr[x][y-1]
            s-=arr[x][y]*arr[x-1][y]
    return s


x=np.random.randint(0,2,[10,8,8])*2-1


# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
H_nonp(x[0])
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))



# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
H_nonp(x[0])
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))


#NOW DO A REFERENCE WITH THE NON JIT

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
H(x[0])
end = time.time()
print("Elapsed (Reference) = %s" % (end - start))


print([H(a) for a in x],[H_nonp(a) for a in x])


Elapsed (with compilation) = 0.23542094230651855
Elapsed (after compilation) = 0.0
Elapsed (Reference) = 0.0010209083557128906
[0, 4, 4, -8, 8, 12, -12, -8, 0, 4] [0, 4, 4, -8, 8, 12, -12, -8, 0, 4]


# Ising MCMC with numba

In [15]:

@jit(nopython=True)
def domain(n,q):
    """Creates a random Ising or potts grid as specified,
    Potts grids are an integer representing which angle and need to be formatted for the rnn but not the hamiltonians"""
    return np.random.randint(0,2,(N,N))*2-1#[[np.random.randint(0,2)*2-1 for a in range(n)]for b in range(n)]#ising
@jit(nopython=True)
def dh(shape,arr,i,j,dq=None):
    y,x = shape
    d=arr[i-1][j]
    u=arr[(i+1)%y][j]
    l=arr[i][j-1]
    r=arr[i][(j+1)%x]
    return arr[i][j]*(d+u+l+r)*2
@jit(nopython=True)
def flipsite(shape):
    """Spin flip where a random spin in the grid is changed (Potts is still an integer representation)"""
    x,y = shape
    x_ = np.random.randint(0,x)
    y_ = np.random.randint(0,y)
    return x_,y_
@jit(nopython=True)
def metropolis_M(x,y,B,iterate = 2500,keep=0.2,seed=5):
   """Runs a monte carlo simulation with the specified hamiltonian function H"""
   np.random.seed(seed)
   C = np.e**(-B)
   arr = domain(x,y)
   shape=(x,y)
   size=x*y
   m= np.sum(arr)//2+size//2
   ms = np.zeros(size+1)
   counter=0
   for i in range(iterate):
       site=flipsite(shape)
       dH = dh(shape,arr,*site)
       n=np.random.random()
       if dH <=0 or n < C**dH:
           arr[site[0]][site[1]]*=-1
           m+= arr[site[0]][site[1]]
       if i>iterate*(1-keep):
           ms[m]=ms[m]+1
       if i>counter:
          counter+=iterate*0.1
          print((i-1)//N**2)
   return ms

@jit(nopython=True)
def H(arr):
    s=0
    w=len(arr)
    h=len(arr[0])
    for x in range(w):
        for y in range(h):
            s-=arr[x][y]*arr[x][y-1]
            s-=arr[x][y]*arr[x-1][y]
    return s
@jit(nopython=True)
def U(B,iterate = 2500,keep=0.2,seed=5):
   """Runs a monte carlo simulation with the specified hamiltonian function H"""
   np.random.seed(seed)
   iterate=int(iterate)
   C = np.e**(-B)
   arr = domain(N,N)
   shape=(N,N)
   size=N*N
   U=H(arr)/size
   U_t=0
   ct=0
   counter=0
   for i in range(iterate):
       site=flipsite(shape)
       dH = dh(shape,arr,*site)
       n=np.random.random()
       if dH <=0 or n < C**dH:
           arr[site[0]][site[1]]*=-1
           U+=dH/size
       if i>iterate*(1-keep):
           U_t+=U
           ct+=1
       if i>counter:
          counter+=iterate*0.1
          print((i-1)//N**2)
   print(U_t/ct)
   return U_t/ct
#@jit(nopython=True)
def M(B,steps,seed=5):
   res=[] 
   eq = metropolis_M(N,N,B,int(steps),seed=seed)
   n=np.sum(eq)
   avg = np.sum([abs(i-N**2/2)/(N**2/2)*eq[i] for i in range(len(eq))])/n
   st_dv = np.power(np.sum([eq[i]*(abs(i-N**2/2)/(N**2/2)-avg)**2 for i in range(len(eq))])/n,0.5)
   res+=[avg]
   res+=[st_dv]
   print(res)
   return res


N=24
B=0.5407
t=time.time()
M(B,N**2*2e5)
print(time.time()-t)
U(B,N**2*2e4)
print(time.time()-t)

0
20000
40000
60000
80000
100000
120000
140000
160000
180000
[0.9485583668174926, 0.02134467779408273]
8.74431824684143
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
-1.8392830128055482
9.962179183959961


In [16]:
a=[]
b=[]
n=16
N=24
B=0.4407
for i in range(n):
    r=U(B,N**2*2e6,seed=i)
    a+=[r]
avg = np.sum(a)/n
dev = np.power(np.sum([(i-avg)**2 for i in a])/n,0.5)
exp = np.math.ceil(-np.log10(dev))
avg=round(avg,exp)
dev=round(dev,exp)
print(a)
print("%s ± %s"%(avg,dev))

0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
-1.4408434487263695
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
-1.4418495099309925
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
-1.4392704642983845
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
-1.4388273792920148
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
-1.4340659142801302
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
-1.4398384260249693
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
-1.440285158425595
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
-1.439869954859967
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
-1.4402762386166348
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
-1.439778228720883
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
-1.4410889026215894
0
200000
4000

In [7]:
a=[]
b=[]
n=16
B=0.5407
for i in range(n):
    r=M(B,N**2*2e6,seed=i)
    a+=[r[0]]
    b+=[r[1]]
avg = np.sum(a)/n
dev = np.power(np.sum([(i-avg)**2 for i in a])/n,0.5)
exp = np.math.ceil(-np.log10(dev))
avg=round(avg,exp)
dev=round(dev,exp)
print(a)
print("%s ± %s"%(avg,dev))

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
[0.9486252369129596, 0.021217009024066473]
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
[0.946457643982388, 0.024910852458317978]
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
[0.947708874270827, 0.021373072948119357]
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
[0.9488310083660818, 0.020509901890065983]
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
[0.948632987627647, 0.021286115154876575]
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
[0.9498285417730941, 0.020350670888591806]
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
[0.949942709212692, 0.020522848503849004]
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
[0.9467390312452393, 0.0227838040358644]
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
[0.9490735380696298, 0.02154365803540699]
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
[0.9490049390453535, 0.021177325549608682]
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
[0.94

In [7]:
-0.556939354313861*8**2

-35.6441186760871

In [ ]:
a=[]
b=[]
N=8
n=16
B=0.25
for i in range(n):
    r=M(B,N**2*2e6,seed=i)
    a+=[r[0]]
    b+=[r[1]]
avg = np.sum(a)/n
st_dv = np.power(np.sum([(i-avg)**2 for i in a])/n,0.5)
print(a)
print(avg,st_dv)

0
22222
44444
66666
88888
111111
133333
155555
177777
200000
[0.3481318704102306, 0.20916694358141694]
0
22222
44444
66666
88888
111111
133333
155555
177777
200000
[0.2559533193243484, 0.17888771554176006]
0
22222
44444
66666
88888
111111
133333
155555
177777
200000
[0.19225591327171537, 0.1419113817940114]
0
22222
44444
66666
88888
111111
133333
155555
177777
200000
[0.21656215799519132, 0.15778775854828134]
0
22222
44444
66666
88888
111111
133333
155555
177777
200000
[0.19426455045376576, 0.14383244273321172]
0
22222
44444
66666
88888
111111
133333
155555
177777
200000
[0.22449590354476184, 0.16394697993742918]
0
22222
44444
66666
88888
111111
133333
155555
177777
200000
[0.2634565618049438, 0.1792610551195109]
0
22222
44444
66666
88888
111111
133333
155555
177777
200000
[0.447333701626588, 0.21937660865178565]
0
22222
44444
66666
88888
111111
133333
155555
177777
200000
[0.5088982417488981, 0.2202284751360471]
0
22222
44444
66666
88888
111111
133333
155555
177777
200000
[0.522383496

In [64]:
def partition_bad(L,B):
    sum_=0
    for a in all_(L):
        sum_+=np.exp(-B*H(np.asarray(a)))
    return sum_

In [75]:
partition_bad(4,0.6)

513383087.8274377

In [66]:
-1/0.4407*np.log(5510942.872396129)/4**2

-2.20136236196772

In [7]:
def partition(L,B):
    c = lambda r: np.cosh(2*B)/np.tanh(2*B)-np.cos(r*np.pi/L)
    y = lambda r: (2*B+np.log(np.tanh(B)) if r==0 else np.log(c(r)+(c(r)**2-1)**0.5))
    Z1=np.prod([2*np.cosh(L/2*y(2*r+1)) for r in range(L)])
    Z2=np.prod([2*np.sinh(L/2*y(2*r+1)) for r in range(L)])
    Z3=np.prod([2*np.cosh(L/2*y(2*r)) for r in range(L)])
    Z4=np.prod([2*np.sinh(L/2*y(2*r)) for r in range(L)])
    return 0.5*(2*np.sinh(2*B))**(0.5*L**2)*(Z1+Z2+Z3+Z4)

In [8]:
def F(L,B):
    return -1/B*np.log(partition(L,B))
def S(L,B,dx=1e-7):
    dx=dx/2
    return (F(L,B+dx)-F(L,B-dx))/(1/(B-dx)-1/(B+dx))

In [14]:
print(F(16,0.4407)/16**2)
print(S(16,0.4407,1e-10)/16**2)
print(F(24,0.4407)/24**2)
print(S(24,0.4407,1e-10)/24**2)

-2.115306343409827
0.29180509699610496
-2.112152537356747
0.2961069308889096


0.2961069308889096